In [1]:
import os
import sqlite3
import pandas as pd

In [2]:
# Set the working directory to the location containing the data
# Note: Insert the path to your working directory
os.chdir(r'C:\Users\Admin\Documents\Programming for Data Science\Practice_03')
os.getcwd()

'C:\\Users\\Admin\\Documents\\Programming for Data Science\\Practice_03'

In [3]:
# Note: Insert your database name - e.g., 'airline_python.db'
conn = sqlite3.connect('airline2.db')
#OperationalError: Could not decode to UTF-8 column 'TailNum' with text '-N600�'--use this for ignoring invalid characters or
#alternatively use the code below to change to latin1 instead
conn.text_factory= lambda x: x.decode('utf-8', 'ignore')
#conn.text_factory = lambda x: str(x, 'latin1')

In [4]:
c = conn.cursor()

In [23]:
# Creating Ontime table, only remove the # for execute and commit if you do not already have a ontime table created in the database
#c.execute('''
CREATE TABLE ontime (
  Year int,
  Month int,
  DayofMonth int,
  DayOfWeek int,
  DepTime  int,
  CRSDepTime int,
  ArrTime int,
  CRSArrTime int,
  UniqueCarrier varchar(5),
  FlightNum int,
  TailNum varchar(8),
  ActualElapsedTime int,
  CRSElapsedTime int,
  AirTime int,
  ArrDelay int,
  DepDelay int,
  Origin varchar(3),
  Dest varchar(3),
  Distance int,
  TaxiIn int,
  TaxiOut int,
  Cancelled int,
  CancellationCode varchar(1),
  Diverted varchar(1),
  CarrierDelay int,
  WeatherDelay int,
  NASDelay int,
  SecurityDelay int,
  LateAircraftDelay int
)
#''')
#conn.commit()

SyntaxError: invalid syntax (981366199.py, line 3)

In [24]:
# Load from csv.bz2 files

# Reading a compressed bz2 file directly is possible by adding "compression='bz2'" argument (see example below)
# x = pd.read_csv("2000.csv.bz2", compression='bz2')

# Alternatively, you could decompress bz2 files into csv then load

#remove # for for loop and commit if there is already a ontime table created beforehand

#for year in range(2000, 2006):
    airline2 = str(year)+".csv.bz2"
    print('Processing:', airline2)
    ontime = pd.read_csv(filename, compression='bz2') #UnicodeDecodeError: 'utf-8' codec can't decode byte 0xe4 in position 343: invalid continuation byte
    ontime = pd.read_csv(filename, encoding="latin-1", low_memory=False, compression='bz2') #avoids UnicodeDecodeError and mixed data type warning for column 22
    ontime = pd.read_csv(filename, encoding="latin-1", compression='bz2') #avoids UnicodeDecodeError
    ontime.to_sql('ontime', con = conn, if_exists = 'append', index = False)

#conn.commit()

IndentationError: unexpected indent (3156469052.py, line 11)

In [14]:
#Q1 – Which plane model has the lowest associated average departure delay
#(excluding cancelled and diverted flights)?

q1= c.execute(''' 
SELECT TailNum, AVG(DepDelay) as avg_depdelay 
FROM ontime 
WHERE Cancelled=0 AND Diverted=0 
GROUP BY TailNum
''').fetchall()

df=pd.DataFrame(q1)

#Pull the row with the least average departure delay
df.columns=['TailNum','AvgDepdelay']
print(df[df.AvgDepdelay==df.AvgDepdelay.min()])

     TailNum  AvgDepdelay
4574  N433AW         -7.5


In [17]:
#Q2 – Which city has the highest number of inbound flights (excluding
#cancelled flights)?
q2= c.execute('''
SELECT Dest, COUNT(Dest) as cityinb
FROM ontime
WHERE Cancelled=0
GROUP BY Dest
''').fetchall()

df2=pd.DataFrame(q2)
df2.columns=['Dest','cityinb']
print(df2[df2.cityinb==df2.cityinb.max()])


    Dest  cityinb
215  ORD  1967069


In [23]:
#Q3 – Which carrier has the highest number of cancelled flights?

q3=c.execute('''
SELECT UniqueCarrier, SUM(Cancelled) as totalcancelled
FROM ontime
WHERE Cancelled>0
GROUP BY UniqueCarrier
''').fetchall()

df3=pd.DataFrame(q3)
df3.columns=['UniqueCarrier','totalcancelled']
print(df3[df3.totalcancelled==df3.totalcancelled.max()])

  UniqueCarrier  totalcancelled
6            DL          107851


In [38]:
#Q4 – Which carrier has the highest number of cancelled flights, relative to
#their number of total flights?

q4=c.execute('''
SELECT UniqueCarrier, SUM(Cancelled) as maxcancelled, COUNT(UniqueCarrier) as totalflight
FROM ontime
GROUP BY UniqueCarrier
''')
df4=pd.DataFrame(q4)
df4.columns=['UniqueCarrier', 'maxcancelled', 'totalflight']
df4['ratio']=df4['maxcancelled']/df4['totalflight']
print(df4[df4.ratio==df4.ratio.max()])

   UniqueCarrier  maxcancelled  totalflight     ratio
12            MQ         96365      2373620  0.040598


In [39]:
conn.close()